In [1]:
CHAPTERS = ['Notes', 'Supervised Learning', 'Unsupervised Learning', 'Statistical Learning']

## Update toc.yml based on the current file structure

In [2]:
import os
import yaml

def get_files(path):
    entries = {}
    with os.scandir(path) as it:
        for entry in it:
            name = entry.name
            name_ext = name.split('.')

            # Skip if the entry is a hidden entry (.*) or a jupyter-book entry (_*)
            if name.startswith('.') or name.startswith('_'):
                continue
                
            if entry.is_file():
                if len(name_ext) > 1 and name_ext[-1] == 'ipynb':
                    entries[name_ext[0]] = None
            else:
                entries[name_ext[0]] = get_files(os.path.join(path, name))
            
    return entries

def files_to_yaml(dirname, files):
    sub_yaml = []
    for key, value in files.items():
        if key.split('.')[0] == 'index':
            continue
        new_dirname = os.path.join(dirname, key)

        entry = {}
        if value is not None:
            sections = files_to_yaml(new_dirname, value)
            if len(sections) == 0:
                continue
            entry['sections'] = sections
        entry['file'] = new_dirname if value is None else os.path.join(new_dirname, 'index')
        entry['title'] = key

        sub_yaml.append(entry)
        
    return sub_yaml

toc_yaml = {
    'format': 'jb-book',
    'root': 'root',
    'parts': [],
}

for chapter in CHAPTERS:
    files = get_files(chapter)
    
    entry = {}
    entry['caption'] = chapter
    entry['chapters'] = files_to_yaml(chapter, files)
    
    toc_yaml['parts'].append(entry)
toc_yaml
    
with open('_toc.yml', 'w') as file:
    yaml.dump(toc_yaml, file)

## Change every notebook's metadata to remove hidden cells

In [3]:
import glob
import nbformat

def change_metadata(path):
    notebook = nbformat.read(path, nbformat.NO_CONVERT)

    changed = False
    for cell in notebook.cells:
        metadata = cell.get('metadata', {})
        tags = set(metadata.get('tags', []))
        jupyter = metadata.get('jupyter', {})

        hiddens = [jupyter.get('source_hidden', False), jupyter.get('output_hidden', False)]
        new_tags = tags - {'hide-cell', 'hide-input', 'hide-output', 'remove-cell', 'remove-input', 'remove-output'}
        if all(hiddens):
            new_tags.add('remove-cell')
        elif hiddens[0]:
            new_tags.add('remove-input')
        elif hiddens[1]:
            new_tags.add('remove-output')

        if new_tags != tags:
            cell['metadata']['tags'] = list(new_tags)
            changed = True

    if changed:
        nbformat.write(notebook, path)
        
for chapter in CHAPTERS:        
    notebook_paths = glob.glob(f'./{chapter}/**/*.ipynb', recursive=True)
    for notebook_path in notebook_paths:
        change_metadata(notebook_path)

## Add last commited date to every notebook

In [4]:
import re
import glob
import nbformat
from git import Repo

def update_date(path):
    # Extract the dates of the first and last commits 
    repo = Repo('./')
    commits = list(repo.iter_commits(paths=path))
    if len(commits) == 0:
        return 
    
    created_time = commits[-1].committed_datetime.strftime('%m-%d-%Y')
    commited_time = commits[0].committed_datetime.strftime('%m-%d-%Y')
    new_source = f'*Created {created_time} (Last commited {commited_time})*',
    
    notebook = nbformat.read(path, nbformat.NO_CONVERT)

    # Define the regex pattern used to search the date cell
    date_regex = '(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])-\d{4}'
    rest_regex = f'^\*(Created ){date_regex}( \(Last commited ){date_regex}\)\*$'
    pattern = re.compile(rest_regex)

    # Find if the date cell already exists
    date_cell = None
    for cell in notebook.cells:
        match = pattern.fullmatch(cell.source) 
        if cell.cell_type == 'markdown' and match != None:
            date_cell = cell
            break

    # Write the date to the file if the date is different or None
    if date_cell == None:
        date_cell = {
            'cell_type': 'markdown',
            'metadata': {},
            'source': new_source,
        }
        notebook.cells.insert(0, nbformat.notebooknode.from_dict(date_cell))
        nbformat.write(notebook, path)
    elif date_cell['source'] != new_source:
        date_cell['source'] = new_source
        nbformat.write(notebook, path)
        
for chapter in CHAPTERS:        
    notebook_paths = glob.glob(f'./{chapter}/**/*.ipynb', recursive=True)
    for notebook_path in notebook_paths:
        update_date(notebook_path)

## Append to conf.py to allow block math

In [5]:
!rm -r _build
!jupyter-book config sphinx .

settings = ['suppress_warnings = ["myst.header"]', 'myst_dmath_double_inline = True']
with open('conf.py', 'a') as file:
    file.write('\n# Belows are more customized settings')
    for setting in settings:
        file.write(f'\n{setting}')
    
!sphinx-build . ./_build/html/ -b html
!rm conf.py

Wrote conf.py to /home/l1qiao/Projects/joeyonng-notebook
Running Sphinx v4.5.0
making output directory... done
[etoc] Changing master_doc to 'root'
[etoc] Excluded 3 extra file(s) not in toc
checking bibtex cache... out of date
parsing bibtex file /home/l1qiao/Projects/joeyonng-notebook/references.bib... parsed 5 entries
myst v0.15.2: MdParserConfig(renderer='sphinx', commonmark_only=False, enable_extensions=['colon_fence', 'dollarmath', 'linkify', 'substitution', 'tasklist'], dmath_allow_labels=True, dmath_allow_space=True, dmath_allow_digits=True, dmath_double_inline=True, update_mathjax=True, mathjax_classes='tex2jax_process|mathjax_process|math|output_area', disable_syntax=[], url_schemes=['mailto', 'http', 'https'], heading_anchors=None, heading_slug_func=None, html_meta=[], footnote_transition=True, substitutions=[], sub_delimiters=['{', '}'], words_per_minute=200)
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 23 source files that are out

## Push the build html pages to gh-pages branch on Github

In [6]:
# If it needs to input the password, run this command in a terminal 
!ghp-import -n -p -f _build/html

Enumerating objects: 38, done.
Counting objects: 100% (38/38), done.
Delta compression using up to 64 threads
Compressing objects: 100% (21/21), done.
Writing objects: 100% (21/21), 31.26 KiB | 4.46 MiB/s, done.
Total 21 (delta 12), reused 0 (delta 0)
remote: Resolving deltas: 100% (12/12), completed with 12 local objects.
To https://github.com/Joeyonng/joeyonng-notebook.git
   4b833eb..168bab3  gh-pages -> gh-pages
